In [11]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import KBinsDiscretizer
from mlxtend.frequent_patterns import apriori, association_rules

In [12]:
n_bins = 2
support = 0.25

In [13]:
### Wczytanie pliku:
path = "../12CPU_30RAM/3600s/A1_100-A2_100-A3_100/merged.csv"
df = pd.read_csv(path).dropna(inplace=False)

#korekcja kolumn:
#merge.csv
if not "merge" in path:
    df = df.iloc[:,1:]
else:
    df = df.iloc[:,2:]

#methods.csv
if "methods" in path:
    endpoint_map = {
        'do-register': 0,
        'add-sell-offer': 1,
        'add-buy-offer': 2,
        'get-stock-data': 3,
        'add-company': 4,
        'get-stock-users-and-companies': 5

    }
    # zastosowanie mapowania na kolumnę endpointUrl
    df['endpointUrl'] = df['endpointUrl'].replace(endpoint_map)

    #
if 'replicaId' in df.columns:
    df['replicaId'] = pd.factorize(df['replicaId'])[0]

#zaokrąglij dane procentowe zużycia do dwóch miejsc po przecinku
cols_to_round = [col for col in df.columns if 'memoryUsage' in col or 'cpuUsage' in col]
if cols_to_round:
    df[cols_to_round] = df[cols_to_round].round(2)


#df = df.drop(["applicationTime", "databaseTime", "queueSizeForward", "trad_databaseTime", "trad_applicationTime"], axis=1)
#unique_endpoints = df['endpointUrl'].unique()
#print(unique_endpoints)


print('Data shape:', df.shape)
print(df.head())


Data shape: (475929, 14)
   apiTime  applicationTime  databaseTime  endpointUrl  queueSizeForward  \
0      480              243           101            0                 0   
1      494              257           108            0                 0   
2      472              244           106            0                 0   
3      483              240            93            0                 0   
4      482              239           102            0                 0   

   queueSizeBack  st_cpuUsage  st_memoryUsage  trad_applicationTime  \
0              0         0.43            0.65                 15850   
1              0         0.43            0.65                 15850   
2              0         0.43            0.65                 15850   
3              0         0.43            0.65                 15850   
4              0         0.43            0.65                 15850   

   trad_databaseTime  trad_numberOfSellOffers  trad_numberOfBuyOffers  \
0              137

In [14]:
### Dyskretyzacja ciągłych wartości w danych:
### Podział każdej z cechy na zbiory o takiej samej liczbie punktów (strategy - quantile)
transformer = KBinsDiscretizer(encode='onehot-dense', n_bins=n_bins)
df_discret = pd.DataFrame(transformer.fit_transform(df).astype(np.int8))

### Utworzenie nowych nazw cech uwzględniających podział otrzymany przy dyskretyzacji:
new_col_names = []

for idx, lst in enumerate(transformer.bin_edges_):
    for i in range(len(lst) - 1):
        new_col_names.append(df.columns[idx] + ": " + str(round(lst[i], 4)) + " - " + str(round(lst[i+1], 4)))
df_discret.columns = new_col_names

### Wyrzucenie zdyskretyzowanych cech o zerowej wariancji:
df_discret = df_discret.loc[:, df_discret.var() > 0]

###
association_rules(apriori(df_discret, min_support=support, use_colnames=True)).sort_values(by='support', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
13,(st_cpuUsage: 0.59 - 0.62),(traf_cpuUsage: 0.59 - 0.62),0.610486,0.729769,0.601833,0.985827,1.350876,0.156320,19.066347
14,(traf_cpuUsage: 0.59 - 0.62),(st_cpuUsage: 0.59 - 0.62),0.729769,0.610486,0.601833,0.824691,1.350876,0.156320,2.221870
8,(databaseTime: 4.0 - 807.0),(applicationTime: 4.0 - 809.0),0.568642,0.602705,0.568642,1.000000,1.659185,0.225918,inf
9,(applicationTime: 4.0 - 809.0),(databaseTime: 4.0 - 807.0),0.602705,0.568642,0.568642,0.943482,1.659185,0.225918,7.632187
18,(trad_applicationTime: 164089.0 - 235789.0),(trad_databaseTime: 152719.0 - 219526.0),0.532096,0.532096,0.532096,1.000000,1.879360,0.248970,inf
...,...,...,...,...,...,...,...,...,...
329,"(apiTime: 29708.0 - 58387.0, databaseTime: 4.0...","(queueSizeBack: 5838.0 - 13407.0, applicationT...",0.255263,0.271364,0.251210,0.984122,3.626574,0.181941,45.888959
330,"(apiTime: 29708.0 - 58387.0, applicationTime: ...","(databaseTime: 4.0 - 807.0, queueSizeBack: 583...",0.272547,0.253945,0.251210,0.921712,3.629566,0.181998,9.529589
331,"(databaseTime: 4.0 - 807.0, queueSizeBack: 583...","(apiTime: 29708.0 - 58387.0, applicationTime: ...",0.253945,0.272547,0.251210,0.989227,3.629566,0.181998,67.526862
332,"(queueSizeBack: 5838.0 - 13407.0, applicationT...","(apiTime: 29708.0 - 58387.0, databaseTime: 4.0...",0.271364,0.255263,0.251210,0.925730,3.626574,0.181941,10.027398
